In [12]:
import pandas as pd
import numpy as np

In [13]:
title_basics = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz', low_memory=False, sep='\t')
title_basics.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0045537,tt0053137,tt0072308,tt0050419"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0037382,tt0038355,tt0071877"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0056404,tt0054452,tt0049189,tt0057345"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0077975,tt0080455,tt0072562,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0060827,tt0083922,tt0050976,tt0050986"


In [14]:
akas = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', low_memory=False, sep='\t')
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [15]:
title_ratings = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', low_memory=False, sep='\t')
title_ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1960
1,tt0000002,5.8,263
2,tt0000003,6.5,1799
3,tt0000004,5.6,179
4,tt0000005,6.2,2597


# Title_basic cleaning process

1. Replace "\N" with np.nan

In [16]:
# Replacing all string values that contains \N with numpy NaN
title_basics = title_basics.replace({'\\N': np.nan})
title_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12385151 entries, 0 to 12385150
Data columns (total 6 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   nconst             object
 1   primaryName        object
 2   birthYear          object
 3   deathYear          object
 4   primaryProfession  object
 5   knownForTitles     object
dtypes: object(6)
memory usage: 566.9+ MB


2. Eliminate movies that are null for runtimeMinutes
3. Eliminate movies that are null for genre

In [17]:
# Dropping all rows with null values in column runtimeMinutes and genres
title_basics.dropna(subset=['runtimeMinutes', 'genres'], inplace=True)

KeyError: ['runtimeMinutes', 'genres']